In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackathon/CreditScore_test.csv
/kaggle/input/hackathon/CreditScore_train.csv
/kaggle/input/hackathon/credit_test.csv
/kaggle/input/hackathon/credit_train.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
import shap
from sklearn.model_selection import train_test_split

# Load datasets
dataset1 = pd.read_csv('/kaggle/input/hackathon/CreditScore_train.csv')

In [3]:
dataset1["Y"] = np.where(dataset1["y"]>800, 1, 0)
print(dataset1["Y"].value_counts())

dataset1=dataset1.drop(['y'],axis=1)
dataset1.fillna(-99999, inplace=True)

train, test = train_test_split(dataset1, test_size=0.3, random_state=444)

y=train["Y"]
X=train.drop(["Y"],axis=1)

y_test=test["Y"]
X_test=test.drop(["Y"],axis=1)

0    71760
1     8240
Name: Y, dtype: int64


In [4]:
# 1- SHAP

model = XGBClassifier()
model.fit(X, y)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
#shap.summary_plot(shap_values, X)
shap_sum = np.abs(shap_values).sum(axis=0)
shap_order = shap_sum.argsort()[::-1]
ordered_columns = X.columns[shap_order]
shap_list=ordered_columns.tolist()
shap_top_20=shap_list[:20]
#shap_top_20

Final_features = pd.DataFrame({'shap': shap_top_20})
Final_features

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


,shap
0,x059
1,x005
2,x275
3,x036
4,x235
5,x020
6,x057
7,x025
8,x181
9,x041


In [5]:
# 2- ADABOOST
# AdaBoost
model = AdaBoostClassifier()
model.fit(X, y)
importance_scores = model.feature_importances_
#top_ab_features = dataset1.columns[np.argsort(importance_scores)[::-1]][:5]
threshold = 0.04
mask = importance_scores >= threshold
columns_above_threshold = X.columns[mask]
scores_above_threshold = importance_scores[mask]
result = pd.DataFrame({'Feature': columns_above_threshold, 'Importance Score': scores_above_threshold})
result = result.sort_values(by='Importance Score', ascending=False)

adaboost_list=result["Feature"].tolist()
ada = pd.DataFrame({'ada': adaboost_list})

Final_features = pd.concat([Final_features, ada], axis=1)

In [6]:
# 3- XGBoost
model = XGBClassifier()
model.fit(X, y)
importance_scores = model.feature_importances_
importance_scores

threshold = 0.04

mask = importance_scores >= threshold
columns_above_threshold = X.columns[mask]
scores_above_threshold = importance_scores[mask]
result = pd.DataFrame({'Feature': columns_above_threshold, 'Importance Score': scores_above_threshold})
result = result.sort_values(by='Importance Score', ascending=False)
xgbboost_list=result["Feature"].tolist()
xgb = pd.DataFrame({'XGB': xgbboost_list})

Final_features = pd.concat([Final_features, xgb], axis=1)
Final_features


,shap,ada,XGB
0,x059,x014,x236
1,x005,x236,x059
2,x275,x005,x181
3,x036,x233,x057
4,x235,x235,x235
5,x020,x015,x076
6,x057,x020,x249
7,x025,x036,NaN
8,x181,x272,NaN
9,x041,NaN,NaN


In [7]:
# 4- random Forest

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X, y)
importance_scores = model.feature_importances_
result = pd.DataFrame({'Feature': X.columns, 'Importance Score': importance_scores})
result = result.sort_values(by='Importance Score', ascending=False)
print(result)
threshold = 0.02

mask = importance_scores >= threshold
columns_above_threshold = X.columns[mask]
scores_above_threshold = importance_scores[mask]
result = pd.DataFrame({'Feature': columns_above_threshold, 'Importance Score': scores_above_threshold})
result = result.sort_values(by='Importance Score', ascending=False)
Rf_list=result["Feature"].tolist()
rf = pd.DataFrame({'RF': Rf_list})

Final_features = pd.concat([Final_features, rf], axis=1)
Final_features

    Feature  Importance Score
56     x057          0.052735
40     x041          0.050930
4      x005          0.034419
234    x235          0.033985
232    x233          0.033147
..      ...               ...
155    x156          0.000000
59     x060          0.000000
132    x133          0.000000
66     x067          0.000000
144    x145          0.000000

[304 rows x 2 columns]


,shap,ada,XGB,RF
0,x059,x014,x236,x057
1,x005,x236,x059,x041
2,x275,x005,x181,x005
3,x036,x233,x057,x235
4,x235,x235,x235,x233
5,x020,x015,x076,x043
6,x057,x020,x249,x236
7,x025,x036,NaN,x239
8,x181,x272,NaN,x044
9,x041,NaN,NaN,x234


In [36]:
# 5- Lasso

from sklearn.linear_model import Lasso
import pandas as pd

model = Lasso(alpha=0.1)  # alpha is the regularization strength
model.fit(X, y)

importance_scores = model.coef_  # obtain the feature importance scores

result = pd.DataFrame({'Feature': X.columns, 'Importance Score': importance_scores})
result = result.sort_values(by='Importance Score', ascending=False)
threshold = 0.01
mask = importance_scores >= threshold
columns_above_threshold = X.columns[mask]
scores_above_threshold = importance_scores[mask]

result = pd.DataFrame({'Feature': columns_above_threshold, 'Importance Score': scores_above_threshold})
result = result.sort_values(by='Importance Score', ascending=False)

lasso_list = result["Feature"].tolist()
lasso = pd.DataFrame({'Lasso': lasso_list})

Final_features = pd.concat([Final_features, lasso], axis=1)
Final_features


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.488e+03, tolerance: 5.238e-01


,shap,ada,XGB,RF,Lasso,Correlation,Lasso
0,x059,x014,x236,x057,x243,x001,NaN
1,x005,x236,x059,x041,x235,x002,NaN
2,x275,x005,x181,x005,x236,x005,NaN
3,x036,x233,x057,x235,x057,x006,NaN
4,x235,x235,x235,x233,x237,x007,NaN
...,...,...,...,...,...,...,...
270,NaN,NaN,NaN,NaN,x119,NaN,NaN
271,NaN,NaN,NaN,NaN,x120,NaN,NaN
272,NaN,NaN,NaN,NaN,x121,NaN,NaN
273,NaN,NaN,NaN,NaN,x122,NaN,NaN


In [37]:
result

,Feature,Importance Score


In [35]:
# 6 Correlation
corr_matrix = X.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.7)]
X1 = X.drop(columns=to_drop)

corr_list = X1.columns.tolist()
corr = pd.DataFrame({'Correlation': corr_list})

Final_features = pd.concat([Final_features, corr], axis=1)
Final_features


`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


,shap,ada,XGB,RF,Lasso,Correlation
0,x059,x014,x236,x057,x243,x001
1,x005,x236,x059,x041,x235,x002
2,x275,x005,x181,x005,x236,x005
3,x036,x233,x057,x235,x057,x006
4,x235,x235,x235,x233,x237,x007
...,...,...,...,...,...,...
270,NaN,NaN,NaN,NaN,x119,NaN
271,NaN,NaN,NaN,NaN,x120,NaN
272,NaN,NaN,NaN,NaN,x121,NaN
273,NaN,NaN,NaN,NaN,x122,NaN


In [38]:
# 7 VIF

from statsmodels.stats.outliers_influence import variance_inflation_factor
df = pd.DataFrame(X, columns=X.columns)
vif = pd.DataFrame()
vif["Feature"] = df.columns
vif["VIF Score"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
vif = vif.sort_values(by='VIF Score', ascending=False)
print(vif)

divide by zero encountered in double_scalars


KeyboardInterrupt: 

In [ ]:
vif = vif.sort_values(by='VIF Score', ascending=False)

# remove features with VIF score greater than 10
top_features = vif[vif['VIF Score'] > 10]['Feature'].values
if len(top_features) > 0:
    print(f"Removing features {', '.join(top_features)} with VIF score greater than 10")
    df = df.drop(columns=top_features)
else:
    print("No features with VIF score greater than 10 found")

# print remaining features with VIF scores
print("Remaining features and their VIF scores:")
print(df)

In [ ]:
!pip install boruta


from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest Classifier model
rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1, class_weight='balanced', max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=42)
boruta_selector.fit(X.values, y.values)

selected_features = X.columns[boruta_selector.support_].tolist()
print(selected_features)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	304
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	304
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	304
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	304
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	304
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	304
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	304
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	155
Tentative: 	63
Rejected: 	86
Iteration: 	9 / 100
Confirmed: 	155
Tentative: 	63
Rejected: 	86
Iteration: 	10 / 100
Confirmed: 	155
Tentative: 	63
Rejected: 	86
Iteration: 	11 / 100
Confirmed: 	155
Tentative: 	63
Rejected: 	86
Iteration: 	12 / 100
Confirmed: 	158
Tentative: 	46
Rejected: 	100
